In [58]:
import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras.layers import *
from keras.models import *
from keras.optimizers import *
from sklearn.decomposition import PCA
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [28]:
features_dr = 'C:/Users/norma/Desktop/BITIRME/GSE_OrtakData_NormR.csv'
labels_dr = 'C:/Users/norma/Desktop/BITIRME/gse_less_feature.csv'

features_df = pd.read_csv(features_dr, sep = ";")
labels_df = pd.read_csv(labels_dr, sep = ",", dtype='unicode')

features = features_df.drop('Class',axis=1)
labels = labels_df.iloc[:,:3]
labels_without_mci = labels_df.iloc[:,:2]

# **PCA**

## n_components = 5

In [29]:
pca = PCA(n_components = 5).fit(features)
principalComponents = pca.fit_transform(features)
columns = ['Pc' + str(i) for i in range(1,6)]
principalDf = pd.DataFrame(data = principalComponents
             , columns = columns)

## n_components = 114

In [30]:
features_df = pd.read_csv(features_dr, sep = ";")
features2 = features_df.drop('Class',axis=1)

pca2 = PCA(n_components = 114).fit(features2)
principalComponents2 = pca2.fit_transform(features2)
columns2 = ['Pc' + str(i) for i in range(1,115)]
principalDf2 = pd.DataFrame(data = principalComponents2
             , columns = columns2)

# **Define CNN Model**

In [31]:
Batch_size = 32
def build_model(TIME_PERIODS):
    input_shape=(TIME_PERIODS,)
    model = Sequential()
    model.add(keras.Input(shape=(TIME_PERIODS,1)))
    model.add(Conv1D(16, 8,strides=2,input_shape=(TIME_PERIODS,1)))
    model.add(Activation('relu'))
    model.add(Conv1D(16, 8,strides=2, kernel_initializer='glorot_normal', padding="same"))
    model.add(Activation('relu'))

    model.add(Conv1D(64, 4,strides=2, kernel_initializer='glorot_normal', padding="same"))
    model.add(Activation('relu'))
    model.add(Conv1D(64, 4,strides=2, kernel_initializer='glorot_normal', padding="same"))
    model.add(Activation('relu'))
    model.add(Conv1D(256, 4,strides=2, kernel_initializer='glorot_normal', padding="same"))
    model.add(Activation('relu'))
    model.add(Conv1D(256, 4,strides=2, kernel_initializer='glorot_normal', padding="same"))
    model.add(Activation('relu'))
    model.add(Conv1D(512, 2,strides=1, kernel_initializer='glorot_normal', padding="same"))
    model.add(Activation('relu'))
    model.add(Conv1D(512, 2,strides=1, kernel_initializer='glorot_normal', padding="same"))
    model.add(Activation('relu'))
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(GlobalAveragePooling1D())
    model.add(Dropout(0.3))
    model.add(Dense(3))
    model.add(Activation('sigmoid'))
    return(model)

# Training with/without MCI class and when n_components = 114

## With MCI when n_components=114

In [32]:
x_train_n114_w, x_test_n114_w, y_train_n114_w, y_test_n114_w = train_test_split(principalDf2, labels, test_size= 0.2, random_state=42) 

In [33]:
y_train_n114_w = y_train_n114_w.astype(int)
y_test_n114_w = y_test_n114_w.astype(int)

In [34]:
ckpt = keras.callbacks.ModelCheckpoint(
        filepath='C:/Users/norma/Desktop/BITIRME/Models/CNN/best_model.{epoch:02d}-{accuracy:.4f}.h5',
        monitor='accuracy', save_best_only=True,verbose=1)

callbacks = [EarlyStopping(monitor='accuracy', patience=30, mode='min', min_delta=0.0001),
         ckpt]

model_n114w = build_model(principalDf2.shape[1])
opt = tf.keras.optimizers.Adam(0.0001)
model_n114w.compile(loss='categorical_crossentropy',
            optimizer=opt, metrics=['accuracy'])

In [35]:
history_n114w = model_n114w.fit(
      x= x_train_n114_w,
      y = y_train_n114_w,
      steps_per_epoch=len(principalDf2)//Batch_size,
      epochs=50,
      callbacks=[callbacks]
      )

y_pred_n114w = model_n114w.predict(x_test_n114_w)
y_pred_n114w = np.argmax(y_pred_n114w, axis=1)

class_d = 'C:/Users/norma/Desktop/BITIRME/GSE_OrtakData_NormR.csv'
class_df = pd.read_csv(class_d, sep = ';')
#creating instance of one-hot-encoder
encoder = OneHotEncoder(handle_unknown='ignore')

#perform one-hot encoding on 'team' column 
encoder_df = pd.DataFrame(encoder.fit_transform(class_df[['Class']]).toarray())
encoder_df.columns = ['AD','CTL','MCI']

y_test_n114_ = encoder.inverse_transform(y_test_n114_w).ravel()
y_test_n114_w = []
for i in y_test_n114_:
    if i == 'MCI':
        y_test_n114_w.append(2)
    elif i == 'AD':
        y_test_n114_w.append(0)
    elif i == 'CTL':
        y_test_n114_w.append(1)

Epoch 1/50
39/39 [==============================] - ETA: 0s - loss: 1.0962 - accuracy: 0.3746
Epoch 1: accuracy improved from -inf to 0.37463, saving model to C:/Users/norma/Desktop/BITIRME/Models\best_model.01-0.3746.h5
39/39 [==============================] - 2s 44ms/step - loss: 1.0962 - accuracy: 0.3746
Epoch 2/50
39/39 [==============================] - ETA: 0s - loss: 1.0855 - accuracy: 0.3726
Epoch 2: accuracy did not improve from 0.37463
39/39 [==============================] - 1s 26ms/step - loss: 1.0855 - accuracy: 0.3726
Epoch 3/50
38/39 [============================>.] - ETA: 0s - loss: 1.0847 - accuracy: 0.3532
Epoch 3: accuracy did not improve from 0.37463
39/39 [==============================] - 1s 28ms/step - loss: 1.0840 - accuracy: 0.3558
Epoch 4/50
38/39 [============================>.] - ETA: 0s - loss: 1.0825 - accuracy: 0.3826
Epoch 4: accuracy improved from 0.37463 to 0.38256, saving model to C:/Users/norma/Desktop/BITIRME/Models\best_model.04-0.3826.h5
39/39 [==

Epoch 29/50
39/39 [==============================] - ETA: 0s - loss: 0.7549 - accuracy: 0.6789
Epoch 29: accuracy improved from 0.67393 to 0.67889, saving model to C:/Users/norma/Desktop/BITIRME/Models\best_model.29-0.6789.h5
39/39 [==============================] - 1s 28ms/step - loss: 0.7549 - accuracy: 0.6789
Epoch 30/50
38/39 [============================>.] - ETA: 0s - loss: 0.6776 - accuracy: 0.7227
Epoch 30: accuracy improved from 0.67889 to 0.72448, saving model to C:/Users/norma/Desktop/BITIRME/Models\best_model.30-0.7245.h5
39/39 [==============================] - 1s 33ms/step - loss: 0.6777 - accuracy: 0.7245
Epoch 31/50
38/39 [============================>.] - ETA: 0s - loss: 0.6484 - accuracy: 0.7389
Epoch 31: accuracy improved from 0.72448 to 0.73835, saving model to C:/Users/norma/Desktop/BITIRME/Models\best_model.31-0.7384.h5
39/39 [==============================] - 2s 39ms/step - loss: 0.6495 - accuracy: 0.7384
Epoch 32/50
38/39 [============================>.] - ETA: 

In [36]:
print(accuracy_score(y_test_n114_w,y_pred_n114w))

0.466403162055336


In [60]:
# confusion matrix
matrix = confusion_matrix(y_test_n114_w,y_pred_n114w, labels=[2,1,0])
print('Confusion matrix : \n',matrix)

# outcome values order in sklearn
tp, fn, fp, tn = confusion_matrix(y_test_n114_w,y_pred_n114w,labels=[2,1,0]).reshape(-1)
print('Outcome values : \n', tp, fn, fp, tn)

# classification report for precision, recall f1-score and accuracy
matrix = classification_report(y_test_n114_w,y_pred_n114w,labels=[1,0])
print('Classification report : \n',matrix)

Confusion matrix : 
 [[21 13 29]
 [19 35 34]
 [27 13 62]]


ValueError: too many values to unpack (expected 4)

## Without MCI when n_components=114

In [37]:
x_train_n114, x_test_n114, y_train_n114, y_test_n114 = train_test_split(principalDf2, labels_without_mci, test_size= 0.2, random_state=42)

In [38]:
y_train_n114 = y_train_n114.astype(int)
y_test_n114 = y_test_n114.astype(int)

In [39]:
Batch_size = 32
def build_model_2(TIME_PERIODS):
    input_shape=(TIME_PERIODS,)
    model = Sequential()
    model.add(keras.Input(shape=(TIME_PERIODS,1)))
    model.add(Conv1D(16, 8,strides=2,input_shape=(TIME_PERIODS,1)))
    model.add(Activation('relu'))
    model.add(Conv1D(16, 8,strides=2, kernel_initializer='glorot_normal', padding="same"))
    model.add(Activation('relu'))

    model.add(Conv1D(64, 4,strides=2, kernel_initializer='glorot_normal', padding="same"))
    model.add(Activation('relu'))
    model.add(Conv1D(64, 4,strides=2, kernel_initializer='glorot_normal', padding="same"))
    model.add(Activation('relu'))
    model.add(Conv1D(256, 4,strides=2, kernel_initializer='glorot_normal', padding="same"))
    model.add(Activation('relu'))
    model.add(Conv1D(256, 4,strides=2, kernel_initializer='glorot_normal', padding="same"))
    model.add(Activation('relu'))
    model.add(Conv1D(512, 2,strides=1, kernel_initializer='glorot_normal', padding="same"))
    model.add(Activation('relu'))
    model.add(Conv1D(512, 2,strides=1, kernel_initializer='glorot_normal', padding="same"))
    model.add(Activation('relu'))
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(GlobalAveragePooling1D())
    model.add(Dropout(0.3))
    model.add(Dense(2))
    model.add(Activation('sigmoid'))
    return(model)

In [40]:
ckpt = keras.callbacks.ModelCheckpoint(
        filepath='C:/Users/norma/Desktop/BITIRME/Models/CNN/best_model_withoutMCI.{epoch:02d}-{accuracy:.4f}.h5',
        monitor='accuracy', save_best_only=True,verbose=1)

callbacks = [EarlyStopping(monitor='accuracy', patience=30, mode='min', min_delta=0.0001),
         ckpt]

model_n114 = build_model_2(principalDf2.shape[1])
opt = tf.keras.optimizers.Adam(0.0001)
model_n114.compile(loss='categorical_crossentropy',
            optimizer=opt, metrics=['accuracy'])

In [53]:
history_n114 = model_n114.fit(
      x= x_train_n114,
      y = y_train_n114,
      steps_per_epoch=len(principalDf2)//Batch_size,
      epochs=50,
      callbacks=[callbacks]
      )

y_pred_n114 = model_n114.predict(x_test_n114)
y_pred_n114 = np.argmax(y_pred_n114, axis=1)

class_d = 'C:/Users/norma/Desktop/BITIRME/GSE_OrtakData_NormR.csv'
class_df = pd.read_csv(class_d, sep = ';')
#creating instance of one-hot-encoder
encoder = OneHotEncoder(handle_unknown='ignore')

#perform one-hot encoding on 'team' column 
encoder_df = pd.DataFrame(encoder.fit_transform(class_df[['Class']]).toarray())
encoder_df = encoder_df.iloc[:,:2]
encoder_df.columns = ['AD','CTL']

y_test_n114_ = encoder.inverse_transform(y_test_n114).ravel()
y_test_n114 = []
for i in y_test_n114_:
    if i == 'AD':
        y_test_n114.append(0)
    elif i == 'CTL':
        y_test_n114.append(1)

Epoch 1/50
39/39 [==============================] - ETA: 0s - loss: 75537668598923264.0000 - accuracy: 0.4975
Epoch 1: accuracy did not improve from 0.53816
39/39 [==============================] - 1s 26ms/step - loss: 75537668598923264.0000 - accuracy: 0.4975
Epoch 2/50
38/39 [============================>.] - ETA: 0s - loss: 87101687654776832.0000 - accuracy: 0.4808
Epoch 2: accuracy did not improve from 0.53816
39/39 [==============================] - 1s 26ms/step - loss: 86199426695036928.0000 - accuracy: 0.4856
Epoch 3/50
39/39 [==============================] - ETA: 0s - loss: 83731246428979200.0000 - accuracy: 0.5064
Epoch 3: accuracy did not improve from 0.53816
39/39 [==============================] - 1s 25ms/step - loss: 83731246428979200.0000 - accuracy: 0.5064
Epoch 4/50
38/39 [============================>.] - ETA: 0s - loss: 87570440385462272.0000 - accuracy: 0.5172
Epoch 4: accuracy did not improve from 0.53816
39/39 [==============================] - 1s 26ms/step - loss

39/39 [==============================] - 1s 27ms/step - loss: nan - accuracy: 0.6244
Epoch 33/50
39/39 [==============================] - ETA: 0s - loss: nan - accuracy: 0.6244
Epoch 33: accuracy did not improve from 0.62438
39/39 [==============================] - 1s 26ms/step - loss: nan - accuracy: 0.6244
Epoch 34/50
39/39 [==============================] - ETA: 0s - loss: nan - accuracy: 0.6244
Epoch 34: accuracy did not improve from 0.62438
39/39 [==============================] - 1s 27ms/step - loss: nan - accuracy: 0.6244
Epoch 35/50
38/39 [============================>.] - ETA: 0s - loss: nan - accuracy: 0.6235
Epoch 35: accuracy did not improve from 0.62438
39/39 [==============================] - 1s 27ms/step - loss: nan - accuracy: 0.6244
Epoch 36/50
37/39 [===========================>..] - ETA: 0s - loss: nan - accuracy: 0.6289
Epoch 36: accuracy did not improve from 0.62438
39/39 [==============================] - 1s 26ms/step - loss: nan - accuracy: 0.6244
Epoch 37/50
37/

ValueError: Shape of the passed X data is not correct. Expected 3 columns, got 2.

In [ ]:
print(accuracy_score(y_test_n114,y_pred_n114))

In [ ]:
# confusion matrix
matrix = confusion_matrix(y_test_n114_w,y_pred_n114w, labels=[2,1,0])
print('Confusion matrix : \n',matrix)

# outcome values order in sklearn
tp, fn, fp, tn = confusion_matrix(y_test_n114_w,y_pred_n114w,labels=[2,1,0]).reshape(-1)
print('Outcome values : \n', tp, fn, fp, tn)

# classification report for precision, recall f1-score and accuracy
matrix = classification_report(y_test_n114_w,y_pred_n114w,labels=[1,0])
print('Classification report : \n',matrix)